# [Day 20](https://adventofcode.com/2024/day/20)

In [107]:
with open("data.txt", "r") as f:
    walls = []
    floors = []
    for y, line in enumerate(f):
        for x, char in enumerate(line):
            if char == "S":
                start = (x, y)
            elif char == "E":
                end = (x, y)
            elif char == "#":
                walls.append((x, y))

In [112]:
from typing import List, Tuple, Dict
from heapq import heappush, heappop


class Maze:
    def __init__(self, walls: List[Tuple[int, int]], start: Tuple[int, int], end: Tuple[int, int]) -> None:
        self.walls = set(walls)
        self.start = start
        self.end = end
        # calculate width and height of max
        max_x, max_y = map(max, zip(*self.walls))
        self.wide = max_x + 1
        self.tall = max_y + 1

    def __repr__(self) -> str:
        grid = [['.' for _ in range(self.wide)] for _ in range(self.tall)]
        for y in range(self.tall):
            for x in range(self.wide):
                if (x, y) in self.walls:
                    grid[y][x] = "#"
                elif (x, y) == self.start:
                    grid[y][x] = "S"
                elif (x, y) == self.end:
                    grid[y][x] = "E"
        # Flatten the list and join with newline
        result = "\n".join("".join(line) for line in grid)
        return result

    def djikstras(self, start_node: Tuple[int, int]):
        # Calculate the distance from start node to every other node.

        # set the distance to start_node to zero
        distances = {}
        distances[start_node] = 0

        # create a set of visited nodes
        visited = set()

        # predecessors
        predecessors = {}

        # create a priority que
        pq = [(0, start_node)]

        while pq:
            current_distance, current_node = heappop(pq)

            # check if current node is already visited
            if current_node in visited:
                continue

            # add current note to visited
            visited.add(current_node)

            # explore neighbours
            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)] # left, right, up, down
            for dx, dy in directions:
                x, y = current_node
                neighbor = x + dx, y + dy

                if neighbor in self.walls or neighbor in visited:
                    continue

                # calculate the distance to neighbor
                new_distance = current_distance + 1

                # Check if neighbor_node exist in distance or if it is better.
                if neighbor not in distances or new_distance < distances[neighbor]:
                    distances[neighbor] = new_distance
                    predecessors[neighbor] = current_node
                    heappush(pq, (new_distance, neighbor)) # push to que

        return distances, predecessors

    def reconstruct_path(self, 
                         target_node: Tuple[int, int],
                         predecessors: Dict[Tuple[int, int], Tuple[int, int]]) -> List[Tuple[int,int]]:
        path = []
        current = target_node

        while current in predecessors:
            path.append(current)
            current = predecessors[current]
        path.append(current) # Add the start node

        return path

    def is_valid(self, node: Tuple[int, int]) -> bool:
        x, y = node
        return (0 <= x < self.wide and 0 <= y < self.tall)

    def cheat(self):
        # Calculate the distance from end position to all other positions
        distances, predecessors = self.djikstras(self.end)

        # Get optimal path
        path = self.reconstruct_path(self.start, predecessors)

        # Track savings counter
        savings = 0
        pico_seconds = 100

        # Precompute valid two-step jumps
        directions = [(-2, 0), (2, 0), (0, -2), (0, 2)]

        for node in path:
            current_distance = distances[node]

            # Calculate two steps
            for dx, dy in directions:
                x, y = node
                new_node = x + dx, y + dy
                if new_node not in self.walls and self.is_valid(new_node):
                    new_distance = distances[new_node] + 2
                    delta = (current_distance - new_distance)
                    if delta >= pico_seconds:
                        savings += 1
        return savings

## Part 1

In [114]:
maze = Maze(walls=walls, start=start, end=end)
savings = maze.cheat()
print("Answer:", savings)

Answer: 1378


## Part 2